### Imports

In [1]:
import gurobipy as gp
import pandas as pd
import numpy as np
import os

%load_ext jupyternotify

<IPython.core.display.Javascript object>

### Functions

In [2]:
def vprint_factory(verbose: bool = False):
    if verbose:
        return print
    return lambda *x, **y: None

In [3]:
def extract(data, row, fact, maximize=True):
    text = str(data.iloc[row][fact]).strip("mg%?")
    if str(text) == "nan" or len(text) == 0:
        return (maximize - 0.5) * 1_000_000
    return float(text)

In [4]:
def classify_age(a: int, year=2020) -> str:
    a = round(a, 0)
    age_strings_2015 = ["1-3", "4-8", "9-13", "14-18", "19-30", "31-50", "51+"]
    age_strings_2020 = ["2-3", "4-8", "9-13", "14-18", "19-30", "31-50", "51+"]
    age_groups = [tuple([int(age) for age in i.split("-")]) if "-" in i else (int(i.strip("+")), 110) for i in age_strings_2015]
    for i, age in enumerate(age_groups):
        if age[0] <= a <= age[1]:
            if year == 2020:
                return age_strings_2020[i]
            else:
                return age_strings_2015[i]
    else:
        raise ValueError(f"{a} is not a valid age in the {year} guidelines.")

### File Input

In [5]:
def load_ref_files(nutrition_name="Nutritional Facts - Categorized", guidelines_name="Dietary Guidelines", year=2020):
    df = pd.read_excel(nutrition_name + ".xlsm", index_col=0)
    guide = pd.read_excel(guidelines_name + ".xlsx", header=1,
                          index_col=[1, 2], skiprows=[2],
                          sheet_name=f"Dietary Guidelines {year}").drop("Unnamed: 0", axis=1)
    return df, guide

### Requirement Selection

In [6]:
def guide_lookup(gender: str, age: int, guide: pd.DataFrame, columns=[]):
    ff_nutrition_to_guidelines = {"Protein": "Protein (g)",
                                  "Vitamin A %": "Vitamin A (mcg RAEd)",
                                  "Sodium": "Sodium (mg)",
                                  "Total Carbohydrates": "Carbohydrate (g)",
                                  "Dietary Fiber": "Fiber (g)",
                                  "Calories": "Calorie Level Assessed", }
    guidelines_to_ff_nutrition = {value: key for key, value in ff_nutrition_to_guidelines.items()}
    guideline_kcals_to_ff_nutrition = {# "Total lipid (% kcal)": "Total Fat",
                                       "Added Sugars (% kcal)": "Sugars",
                                       "Saturated Fatty Acids (% kcal)": "Saturated Fat",
                                       "Calorie Level Assessed": "Calories From Fat"}
    genders = {"m": "Male", "f": "Female"}
    gender = genders[gender[0].lower()] # 'm' / 'M' / 'male' / 'Male' -> 'Male'
    # Selecting the appropriate Row
    filtered = guide.loc[gender, classify_age(age)]
    # Handling kcal measurements
    cal_level = filtered["Calorie Level Assessed"]
    nutrient_cals = {"Total Fat": 9, "Saturated Fat": 9, "Sugars": 4, "Calories From Fat": cal_level / 10}  # Cals from fat <= 10% cal_level
    for kcal_nutrient, out_name in guideline_kcals_to_ff_nutrition.items():
        new_entry = pd.Series(index=[out_name],
                              data=float(str(filtered[kcal_nutrient]).strip("<>").split("-")[-1]) / 100
                                         * cal_level / nutrient_cals[out_name])
        filtered = filtered.append(new_entry)
        filtered.drop(columns=kcal_nutrient, inplace=True)
    filtered.drop((col for col in filtered.index
                   if col not in guidelines_to_ff_nutrition.keys()
                   and col not in guideline_kcals_to_ff_nutrition.values()),
                  inplace=True)
    filtered.rename(guidelines_to_ff_nutrition, inplace=True) # I made this right at the end to potentially fix a problem, but I don't think it worked. Evaluate!
    return filtered[columns] if columns else filtered

### Model Building: Variables, Constraints, and Objective

In [7]:
def create_model(subset: pd.DataFrame, less_thans: list, guide: pd.DataFrame, costs: dict, objective: list,
                 filter_relaxations: set = {}, meals: int = 2, cat_limit: int = 0, min_cal_cutoff: int = 1,
                 var_type: str = "I", verbose: bool = False):
    """
    Generates the Gurobi model according to many available filters and options, discussed below
    
    :subset: menu items to consider, frequently broken down by restaurant
    :less_thans: list of nutrients in guide where the sum of food nutritions must be less than the guideline (such as sodium) rather than more (protein)
    :guide: dietary recommendations taken from the government's '20-'25 Dietary Guidelines For Americans.
    :costs: dictionary of penalties associated with exceeding / falling short of the recommendations. Must include objective nutrients
    :objective: list of which nutrient(s) to rank by. Multiple items cause a multi-objective
    :filter_relaxations: recommendations in guide which may be relaxed. TODO: untested
    :meals: requires the solution to meet only (1 / meals) of each nutrient recommendation. Meals=1 & cat_limit=1 usually infeasible. Default: 2
    :cat_limit: requires no more than cat_limit of any one food type in the solution. Prevents 31 apple juice box solutions. Default: False (no limit)
    :min_cal_cutoff: when non-zero, omits foods from the solution with fewer calories than the cutoff. Default: 1
    :var_type: decision variable type in B(inary), I(nteger), or C(ontinuous). Default: Integer
    :verbose: provides detailed constraining / solving progress updates. Disable for more concise output. Default: False
    """
    vprint = vprint_factory(verbose)
    
    if type(objective) is not list:
        objective = [objective]
    if len(objective) > 1:
        for obj in objective:
            if obj not in costs.keys():
                raise ValueError(f"{obj} has no associated cost in costs: {costs}. This is required to form a multi-objective.")
        filter_relaxations.update(cost for cost in costs.keys() if cost in objective)  # Multiobjectives rely on relaxations and costs for their variables
    
    m = gp.Model()
    if (vtype := var_type[0].upper()) not in "BIC":
        raise ValueError(f"var_type must be one of B(inary), I(nteger), or C(ontinuous). {vtype} was passed")
    xis = [m.addVar(vtype=vtype) for _ in subset.index]  # Whether to include a food in the meal
    f_rel = {fact: m.addVar(name=f"{fact}_rel") if fact in filter_relaxations else 0 for fact in guide.index}  # excess variable
    
    for fact, req in guide.items():  # Nutrition Requirements
        if len(objective) == 1 and fact == objective:  # Don't constrain the objective for single-objective models
            continue
        if fact in less_thans:
            vprint(f"Constraining {fact}".ljust(35), f"<= {round(req, 2)}".ljust(10), f"across {meals} meals")
            m.addConstr(sum((x * extract(subset, r, fact, maximize=True) for r, x in enumerate(xis))) - f_rel[fact] <= req / meals)
        else:
            vprint(f"Constraining {fact}".ljust(35), f">= {round(req, 2)}".ljust(10), f"across {meals} meals")
            m.addConstr(sum((x * extract(subset, r, fact, maximize=False) for r, x in enumerate(xis))) + f_rel[fact] >= req / meals)

    if min_cal_cutoff:
        for i, food in enumerate(subset.iterrows()):  # Excludes zero calorie (from fat) entries
            if extract(subset, i, "Calories From Fat", maximize=False) < min_cal_cutoff:  # If Calories From Fat are unknown, they're assumed to be 0
                m.addConstr(xis[i] == 0)  # If a food has fewer than the min_cal_cutoff, require 0 of it in the solution
                
    if cat_limit:
        vprint()
        for cat in pd.unique(subset["Common Category"]):
            vprint(f"Constraining only {cat_limit} or fewer {cat.strip('s')} items.")
            m.addConstr(sum((x for i, x in enumerate(xis) if subset["Common Category"][i] == cat)) <= cat_limit)
    
    m.setParam("OutputFlag", verbose)
    
    if len(objective) > 1:
        # The multiobjective value is the sum of the products of each's nutrients overage / shortage with its relative weight
        m.ModelSense = gp.GRB.MINIMIZE
        m.setObjective(sum(difference * costs[nutrient]
                           if type(difference) is gp.Var else 0
                           for nutrient, difference in f_rel.items() if nutrient in objective))
    else:
        # For less_than constraints, the nutrient ought to be minimized.
        # If data for an entry is unknown, it ought to be expensive for minimization
        # and negatively expensive for maximization as as to not be included in the solution
        m.ModelSense = gp.GRB.MINIMIZE if objective[0] in less_thans else gp.GRB.MAXIMIZE
        m.setObjective(sum((x * extract(subset, r, objective[0], maximize= -m.ModelSense) for r, x in enumerate(xis))) / meals)
    return m

In [8]:
def solution_data(model: gp.Model, model_args: dict, columns: list, multi_objective: bool) -> (pd.Series, pd.DataFrame):
    if model.status != 2:
        return None, None
    results = pd.Series(index=model_args["objective"], name=model_args["subset"].iloc[0]["Restaurant"], dtype="float64")
    
    if multi_objective:
        results.loc[model_args["objective"]] = [model.getVarByName(name + "_rel").x for name in model_args["costs"].keys()]
        results = results.append(pd.Series(model.ObjVal, index=["Overage Penalty"]))    
        choices = [var.x > 0 for var in model.getVars() if "_rel" not in var.varName]
        values = [var.x for var in model.getVars() if var.x > 0 and "_rel" not in var.varName]
    else:
        results.iloc[0] = model.ObjVal
        choices = [var.x > 0 for var in model.getVars()]
        values = [var.x for var in model.getVars() if var.x > 0]
    food_choice = model_args["subset"].loc[choices, ["Food"] + columns].reset_index(drop=True)
    food_choice.loc[:, "Amount"] = values
    return results, food_choice

In [15]:
def model_run(df, objectives, multi_obj, res="", verbose=False):
    vprint = vprint_factory(verbose)
    res_list = [res] if res else pd.unique(df["Restaurant"])
    master_results = pd.DataFrame(columns=objectives + (["Overage Penalty"] if multi_obj else []), index=res_list)
    master_foods = pd.DataFrame(columns=columns + ["Restaurant"] + (["Objective"] if not multi_obj else []), index=[])
    
    for res in res_list:
        vprint(res, end=", ")
        subset = df.loc[df["Restaurant"] == res].reset_index(drop=True)
        model_args["subset"] = subset

        if multi_obj:
            m = create_model(**model_args)
            m.optimize()
            results, foods = solution_data(m, model_args, columns, multi_obj)
        else:
            results = pd.DataFrame(columns=[], index=[res])
            foods = pd.DataFrame(columns=columns + ["Objective", "Restaurant"], index=[])
            for obj in objectives:
                model_args["objective"] = [obj]
                m = create_model(**model_args)
                m.optimize()
                new_results, new_foods = solution_data(m, model_args, columns, multi_obj)
                if new_results is None:
                    vprint(f"{res}-{obj} is infeasible.")
                    new_results = pd.Series(index=[obj], name=res, dtype="float64")
                    results = pd.concat([results, pd.DataFrame(new_results).T], axis=1)
                    continue
                results = pd.concat([results, pd.DataFrame(new_results).T], axis=1)
                new_foods.insert(0, "Objective", obj)
                foods = pd.concat([foods, new_foods], axis=0).reset_index(drop=True)
    #             display(results)

        if foods is not None:
            foods.loc[:, "Restaurant"] = res
            master_results.loc[res] = results.iloc[0]
            master_foods = master_foods.append(foods, ignore_index = True)
        else:
            master_results.loc[res] = "-"

    food_col_order = ['Restaurant', 'Amount', 'Food', 'Sodium', 'Sugars', 'Calories From Fat',
                      'Protein', 'Total Carbohydrates', 'Dietary Fiber', 'Saturated Fat']
    if not multi_obj:
        food_col_order.insert(1, "Objective")
    master_foods = master_foods.reindex(columns = food_col_order)

    if multi_obj:
        master_results = master_results.reindex(columns = master_results.columns[[3, 0, 1, 2]])
        master_results = master_results.replace("-", np.nan).sort_values("Overage Penalty")
    master_results.replace(np.nan, "-", inplace=True)

    return master_results, master_foods


### Status
#### Finished

* Understand why various alternate objectives aren't all feasible / infeasible together
* Finish single objective optimization (omitting actual meals, just assessing feasibility)
* Talk about appropriate Calorie / Sugar / Sodium Balancing
* Add ability to introspect any particular model to analyze food choices and relaxations
* Update guide to 2020-2025 recommendations
    * Add conversion dictionary to import function to allow various constraints from new dataset
* Constrain no more than 10% of the calories to come from fat
* Simplify model: trim down to only constraints mentioned in paper
* Drop Vitamin A because so few foods have data
* Work towards general metric capable of ranking restaurants against one another - do for each objective
* Individual rankings (splitting multiobjective)
* Consider adding other relaxations
* Get list of nutrients by which we are actually constraining
* Simplify model to obtain more feasible solutions (ignore vitamins) (use protein, fiber, calories, iron, sugars, sodium, etc.) - potentially unnecessary given recent improvements
* Work with C.C. to display and analyze selected menus
* Multicriteria optimization so that the sum of all three overages is minimized, rather than just one plus the overages of the other two

### Broken

### In Progress
* Check whether data problems are consistent by restaurant or whatever & fix! (Ex. Godfather's Pizza)
* Contact FFN about dataset problems
* Update individual analysis to allow using individual objectives

### Future
* Write all optimal diets out a spreadsheet


### Data Loading

In [12]:
df, guide = load_ref_files()

### Run Config

In [16]:
age, gender = 24, "Male"
objectives = ["Sugars", "Sodium", "Calories From Fat"]  # List of objective(s). Must be list-like
multi_obj = True  # If True, the model objective is the sum of objective overages. Otherwise, the model is solved for each nutrient separately
verbose = False   # Whether to provide run / solve updates
res = "Popeyes"   # Leave blank to run all
out_name = "Bulk" # Leave blank to skip saving

columns = ['Sodium', 'Sugars', 'Calories From Fat', 'Protein',
           'Total Carbohydrates', 'Dietary Fiber', 'Saturated Fat']
filtered_guide = guide_lookup(gender, age, guide, columns)

less_thans = ["Sodium", "Sugars", "Saturated Fat", "Calories From Fat"]  # These correspond to columns from Fast Food Nutrition dataset
costs = {"Sugars": 1, "Sodium": 50, "Calories From Fat": 4}   # Note units: Sugars (g) vs Sodium (mg)
model_args = {"less_thans": less_thans, "guide": filtered_guide, "filter_relaxations": set(), "objective": objectives,
              "costs": costs, "meals": 1, "cat_limit": False, "min_cal_cutoff": 1, "verbose": verbose, }

results, foods = model_run(df, objectives, multi_obj, res, verbose)
display(results)
display(foods)

,Overage Penalty,Sugars,Sodium,Calories From Fat
Popeyes,0.0,0.0,0.0,0.0


,Restaurant,Amount,Food,Sodium,Sugars,Calories From Fat,Protein,Total Carbohydrates,Dietary Fiber,Saturated Fat
0,Popeyes,10.0,Corn On The Cob,0mg,0g,18,6g,37g,4g,0.5g


### Saving Output

In [14]:
outname = "./" + out_name + ".xlsx"
with pd.ExcelWriter("./" + out_name + ".xlsx", mode='a') as writer:
    master_results.to_excel(writer, sheet_name = "Rankings1" + ("-Multi" if multi_obj else ""))
    master_foods.to_excel(writer, sheet_name = "Foods1" + ("-Multi" if multi_obj else ""), index=False)
%notify -m "Results saved"

<IPython.core.display.Javascript object>